In [ ]:
import sys
import os
import random
from torchvision import transforms
from home_robot.agent.multitask.sparse_voxel_instance_map import SparseVoxelMapAgent
from PIL import Image
import numpy as np
from home_robot.datasets.scannet import ScanNetDataset
from home_robot.datasets.scannet.referit3d_data import ReferIt3dDataConfig, load_referit3d_data
from home_robot.datasets.scannet.scanrefer_data import ScanReferDataConfig, load_scanrefer_data
from home_robot.datasets.scannet.scannet_dataset import ScanNetDataset
from home_robot.datasets.scannet.scannet_constants import NUM_CLASSES as NUM_CLASSES_LONG
from home_robot.perception import OvmmPerception
import torch
import hydra
import json
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
# from data_visualization import InstanceMemeoryVisualizer
from typing import List, Optional, Tuple, Union
import cv2
import pickle
from pathlib import Path


In [ ]:

def get_image_from_path(
    image_path: Union[str, Path],
    height: Optional[int] = None,
    width: Optional[int] = None,
    keep_alpha: bool = False,
) -> torch.Tensor:
    """Returns a 3 channel image.
    # Adapted from https://github.com/nerfstudio-project/nerfstudio/blob/main/nerfstudio/data/dataparsers/scannet_dataparser.py
    Args:
        image_idx: The image index in the dataset.
    """
    pil_image = Image.open(image_path)

    assert (height is None) == (width is None)  # Neither or both
    if height is not None:
        pil_image = pil_image.resize((width, height), resample=Image.BILINEAR)
    image = np.array(pil_image, dtype="uint8")  # shape is (h, w) or (h, w, 3 or 4)
    if len(image.shape) == 2:
        image = image[:, :, None].repeat(3, axis=2)
    assert len(image.shape) == 3
    assert image.dtype == np.uint8
    assert image.shape[2] in [3, 4], f"Image shape of {image.shape} is in correct."
    image = torch.from_numpy(image.astype("float32") / 255.0)
    if not keep_alpha and image.shape[-1] == 4:
        image = image[:, :, :3]
        # image = image[:, :, :3] * image[:, :, -1:] + self._dataparser_outputs.alpha_color * (1.0 - image[:, :, -1:])
    return image

def get_depth_image_from_path(
    filepath: Path,
    height: Optional[int] = None,
    width: Optional[int] = None,
    scale_factor: float = 1.0,
    interpolation: int = cv2.INTER_NEAREST,
) -> torch.Tensor:
    """Loads, rescales and resizes depth images.
    Filepath points to a 16-bit or 32-bit depth image, or a numpy array `*.npy`.
    # Adapted from https://github.com/nerfstudio-project/nerfstudio/blob/main/nerfstudio/data/dataparsers/scannet_dataparser.py
    Args:
        filepath: Path to depth image.
        height: Target depth image height.
        width: Target depth image width.
        scale_factor: Factor by which to scale depth image.
        interpolation: Depth value interpolation for resizing.

    Returns:
        Depth image torch tensor with shape [height, width, 1].
    """
    assert (height is None) == (width is None)  # Neither or both
    do_resize = height is not None
    if filepath.suffix == ".npy":
        image = np.load(filepath) * scale_factor
        assert (height is None) == (width is None)  # Neither or both
        if do_resize:
            image = cv2.resize(image, (width, height), interpolation=interpolation)
    else:
        image = cv2.imread(str(filepath.absolute()), cv2.IMREAD_ANYDEPTH)
        image = image.astype(np.float64) * scale_factor
        if do_resize:
            image = cv2.resize(image, (width, height), interpolation=interpolation)
    return torch.from_numpy(image[:, :, np.newaxis])


In [ ]:
class OVMMLoader:
    DEPTH_SCALE_FACTOR = 1000 / (2**16)
    FRAME_SKIP = 1
    HEIGHT = 640
    WIDTH = 480
    def __init__(self, path, height: Optional[int] = 480, width: Optional[int] = 640):
        self.path = path
        self.height = self.HEIGHT
        self.width = self.WIDTH

    def load_scene(self, scene = "000-hm3d-BFRyYbPCCPE"): # testing
        # option 1: TODO: do we still want to load from Arjun's directory?
        scene_path = os.path.join(self.path, scene)
        if not os.path.exists(scene_path):
            return None
        steps = set()
        for file in os.listdir(scene_path):
            if any(char.isdigit() for char in file):
                steps.add(file[:5])
        steps = list(sorted(steps))
        steps = steps[0::self.FRAME_SKIP]
        images = []
        depths = []
        poses = []
        for step in steps:
            rgb_file = os.path.join(scene_path, step + '-rgb.png')
            depth_file = os.path.join(scene_path, step + '-depth.png')
            pose_file = os.path.join(scene_path, step + '.txt')
            rgb_tensor = get_image_from_path(Path(rgb_file), height=self.height, width=self.width)
            depth_tensor = get_depth_image_from_path(
                Path(depth_file),
                height=self.height,
                width=self.width,
                scale_factor=self.DEPTH_SCALE_FACTOR,
            )
            valid_depth = (depth_tensor > 0.1) & (depth_tensor < 4)
            if True not in valid_depth:
                continue
            images.append(rgb_tensor)
            depths.append(depth_tensor)
            # with open(pose_file, 'rb') as f:
            #     pose_info = pickle.load(f)
            poses.append(torch.from_numpy(np.loadtxt(pose_file)))
        intrinsic_file = os.path.join(scene_path, 'intrinsic_color.txt')  
        intrinsic = np.loadtxt(intrinsic_file)  
        intrinsics = torch.from_numpy(np.repeat(np.expand_dims(intrinsic, axis=0), len(steps),  axis=0)).to("cuda").float()
        images = torch.stack(images).float().to("cuda")
        depths = torch.stack(depths).float().to("cuda")
        poses = torch.stack(poses).float().to("cuda")
        scene_obs = {"images": images, "depths": depths, "poses": poses, "intrinsics": intrinsics}
        return scene_obs


In [ ]:


def opengl_to_opencv(pose):
    transform = np.array(
        [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]
    )
    pose = pose @ transform
    return pose

def from_width_height_fov_to_intrinsics(
    width: float,
    height: float,
    fov_degrees: float,
):
    """Create a simple pinhole camera given minimal information only. Fov is in degrees"""
    horizontal_fov_rad = np.radians(fov_degrees)
    h_focal_length = width / (2 * np.tan(horizontal_fov_rad / 2))
    v_focal_length = width / (
        2 * np.tan(horizontal_fov_rad / 2) * float(height) / width
    )
    principal_point_x = (width - 1.0) / 2
    principal_point_y = (height - 1.0) / 2
    # principal_point_x = width / 2
    # principal_point_y = height / 2
    # return np.array([[v_focal_length, 0, principal_point_x, 0],
    #                  [0, h_focal_length, principal_point_y, 0],
    #                  [0, 0, 1, 0],
    #                  [0, 0, 0, 1]])
    return np.array([[h_focal_length, 0, principal_point_x, 0],
                     [0, h_focal_length, principal_point_y, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
# def convert_obs_to_instance_memory(scene_id, scene_obs):
#     if not scene_obs:
#         return
#     print (f'running instance memory on {scene_id}')
#     self.instance_agent.build_scene_and_get_instances_for_queries(scene_obs, queries=[], reset=False)

#     instances = self.instance_agent.voxel_map.get_instances()
#     num_instances = len(instances)
#     print (f"{num_instances} instances perceived")
#     os.makedirs(os.path.join(self.save_dir, scene_id), exist_ok=True)
#     with open(os.path.join(self.save_dir, scene_id, 'voxel_map.pkl'), 'wb') as f:
#         pickle.dump(self.instance_agent.voxel_map, f)
class InstanceEvaluator:
    
    def __init__(self, instance_agent, save_dir=None):
        self.instance_agent = instance_agent
        self.ovmm = OVMMLoader()
        # Add to ScanNetSparseVoxelMap
        class_id_to_class_names = dict(
            zip(
                self.scannet.data.METAINFO["CLASS_IDS"],  # IDs [1, 3, 4, 5, ..., 65]
                self.scannet.data.METAINFO["CLASS_NAMES"],  # [wall, floor, cabinet, ...]
            )
        )
        self.instance_agent.set_vocabulary(class_id_to_class_names)
        self.save_dir = save_dir
        if save_dir:
            self.visualizer = InstanceMemeoryVisualizer(save_dir)



    
    def eval_ovmm_eqa(self):
        """run instance memory pipeline on the annotated ScanNet EQA dataset"""
        # TODO: change to the new annotation
        with open("/private/home/xiaohanzhang/data/ovmm_bench_500_frames/ovmm_bench_v1_annotated.json") as f:
            annotations = json.load(f)
        scenes = set()
        for ann in annotations:
            if 'success' in ann:
                scenes.add(ann['episode_id'])
        scenes = sorted(list(scenes))
        print (scenes)

        for scene_id in scenes:
            if os.path.exists(os.path.join(self.save_dir, scene_id)):
                continue
            self.instance_agent.reset()
            scene_obs = self.ovmm.load_scene(scene=scene_id)
            self.convert_obs_to_instance_memory(scene_id, scene_obs)


In [ ]:
# @hydra.main(config_path="/private/home/xiaohanzhang/home-robot/projects/scannet_offline_eval/configs/model", 
#             config_name="instancemap3d_top_down_detic")
scannet = ScanNetDataset(
    root_dir = '/private/home/ssax/home-robot/src/home_robot/home_robot/datasets/scannet/data',
    frame_skip = 180,
    split = 'val',
    n_classes=NUM_CLASSES_LONG,
    referit3d_config = ReferIt3dDataConfig(),
    scanrefer_config = ScanReferDataConfig(),    
)
from hydra import compose, initialize
from omegaconf import OmegaConf
# context initialization
with initialize(version_base=None, config_path="../scannet_offline_eval/configs/model", job_name="test_app"):
    cfg = compose(config_name="instancemap3d_top_down_detic")
    print(OmegaConf.to_yaml(cfg))
instance_agent = instantiate(cfg)

In [ ]:
# set vocabulary
class_id_to_class_names = dict(
    zip(
        scannet.METAINFO["CLASS_IDS"],  # IDs [1, 3, 4, 5, ..., 65]
        scannet.METAINFO["CLASS_NAMES"],  # [wall, floor, cabinet, ...]
    )
)
instance_agent.set_vocabulary(class_id_to_class_names)

In [ ]:
data_path = "/private/home/xiaohanzhang/accel-cortex/frames"
data_frame_path = data_path+'_frames'
# scene_id = "104348328_171513363_790"
scene_id = ""
print (f'extracting scene id {scene_id}')
with open(os.path.join(data_path, scene_id, 'obs_data.pkl'), 'rb') as f:
    obs = pickle.load(f)
import shutil
if os.path.exists(os.path.join(data_frame_path, scene_id)):
    shutil.rmtree(os.path.join(data_frame_path, scene_id))
os.makedirs(os.path.join(data_frame_path, scene_id), exist_ok=True)


In [ ]:

found_obj = False
found_recep = False
for step in range(len(obs)):
    format_step = "{:05d}".format(step)
    # perceived_ids = np.unique(obs[step].task_observations['gt_instance_ids'])
    # save rgb
    if not os.path.exists(os.path.join(data_frame_path, scene_id, f'{format_step}-rgb.png')):
        print (obs[step].rgb)
        rgb = Image.fromarray(obs[step].rgb)
        rgb.save(os.path.join(data_frame_path, scene_id, f'{format_step}-rgb.png'))
    # # save depth
    if not os.path.exists(os.path.join(data_frame_path, scene_id, f'{format_step}-depth.png')):
        depth = Image.fromarray((obs[step].depth/1000*2**16).astype(np.uint16))
        depth.save(os.path.join(data_frame_path, scene_id, f'{format_step}-depth.png'))        
    # save pose
    # print (obs[step].camera_pose)
    # print (get_real_world_camera_pose(obs[step]))
    # np.savetxt(os.path.join(data_frame_path, scene_id, f'{format_step}.txt'), get_real_world_camera_pose(obs[step]))
    new_pose = obs[step].camera_pose
    # print (new_pose)
    np.savetxt(os.path.join(data_frame_path, scene_id, f'{format_step}.txt'), new_pose)
        
# save_intrinsic
# TODO: uncomment this
np.savetxt(os.path.join(data_frame_path, scene_id, 'intrinsic_color.txt'), from_width_height_fov_to_intrinsics(480, 640, 42))
np.savetxt(os.path.join(data_frame_path, scene_id, 'intrinsic_depth.txt'), from_width_height_fov_to_intrinsics(480, 640, 42))

ovmm = OVMMLoader(path=data_frame_path)
scene_obs = ovmm.load_scene(scene=scene_id)
np.set_printoptions(suppress=True)
# print (np.round(scene_obs['poses'][0].cpu().numpy(), decimals=2))
print (scene_obs['poses'].cpu().numpy())
from home_robot.datasets.eqa.utils import relative_transformation, to_scalar
def _preprocess_poses(poses: torch.Tensor):
    r"""Preprocesses the poses by setting first pose in a sequence to identity and computing the relative
    homogenous transformation for all other poses.

    Args:
        poses (torch.Tensor): Pose matrices to be preprocessed

    Returns:
        Output (torch.Tensor): Preprocessed poses

    Shape:
        - poses: :math:`(L, 4, 4)` where :math:`L` denotes sequence length.
        - Output: :math:`(L, 4, 4)` where :math:`L` denotes sequence length.
    """
    return relative_transformation(
        poses[0].unsqueeze(0).repeat(poses.shape[0], 1, 1),
        poses,
        orthogonal_rotations=False,
    )
relative_pose = False
poses = scene_obs['poses']
print (len(poses))
start, end, stride = 0, -1, 1
if end == -1:
    end = len(poses)
poses = poses[start : end : stride]
print (len(poses))
# self.transformed_poses = datautils.poses_to_transforms(self.poses)
# poses = torch.stack(poses)
if relative_pose:
    transformed_poses = _preprocess_poses(poses)
else:
    transformed_poses = poses
print (transformed_poses)
scene_obs['poses'] = transformed_poses


# pose1 = scene_obs['poses'][0]
# pose2 = scene_obs['poses'][2]

In [ ]:
instance_agent.reset()
instance_agent.build_scene_and_get_instances_for_queries(scene_obs, queries=[], reset=False)
instances = instance_agent.voxel_map.get_instances()
num_instances = len(instances)
print (f"{num_instances} instances perceived")
instance_agent.voxel_map.show(backend="pytorch3d")
# os.makedirs(os.path.join(self.save_dir, scene_id), exist_ok=True)
# with open(os.path.join(self.save_dir, scene_id, 'voxel_map.pkl'), 'wb') as f:
#     pickle.dump(self.instance_agent.voxel_map, f)

In [ ]:
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_scene, get_camera_wireframe
from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from home_robot.utils.point_cloud_torch import unproject_masked_depth_to_xyz_coordinates
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def colormap_to_rgb_strings(data, colormap_name='viridis', include_alpha=False, min_val=None, max_val=None):
    """
    Convert a range of numbers from a given dataset into a series of RGB or RGBA strings using a specified Matplotlib colormap.

    :param data: The dataset from which to derive color mappings.
    :param colormap_name: The name of the Matplotlib colormap to use.
    :param include_alpha: Boolean to decide if the alpha channel should be included in the RGB strings.
    :param min_val: Optional minimum value for colormap scaling.
    :param max_val: Optional maximum value for colormap scaling.
    :return: A list of color strings in the format 'rgb(R,G,B)' or 'rgba(R,G,B,A)'.
    """
    # Compute min and max from the data if not provided
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)

    # Normalize data within the provided or computed min and max range
    norm = plt.Normalize(min_val, max_val)
    colors = plt.cm.get_cmap(colormap_name)(norm(data))

    # Format color strings based on the include_alpha flag
    if include_alpha:
        return ["rgba({},{},{},{})".format(int(r*255), int(g*255), int(b*255), a) for r, g, b, a in colors]
    else:
        return ["rgb({},{},{})".format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors[:, :3]]
def add_camera_poses(
    fig,
    poses,
    linewidth = 3,
    color = None,
    name = 'cam',
    separate = True,
    scale = 0.2,
    colormap_name='plasma'
    ):
    cam_points = get_camera_wireframe(scale)
    # Convert p3d (opengl) to opencv
    cam_points[:, 1] *= -1

    if color is None:
        colors = colormap_to_rgb_strings(list(range(len(poses))), colormap_name=colormap_name)
    else:
        colors = [color] * len(poses)
    for i, (pose, color) in enumerate(zip(poses, colors)):
        # cam_points[:, 2] *= -1
        R = pose[:3, :3]
        t = pose[:3, -1]
        cam_points_world = cam_points @ R.T + t.unsqueeze(0)  # (cam_points @ R) # + t)
        x, y, z = [v.cpu().numpy().tolist() for v in cam_points_world.unbind(1)]
        fig.add_trace(
            go.Scatter3d(
                x=x,
                y=y,
                z=z,
                mode="lines",
                marker={
                    "size": 1,
                    "color": color,
                },
                line=dict(
                    width=linewidth,
                    color=color,
                ),
                name=f'{name}-{i}',
            )
        )

rgb, depth, pose, Ks = scene_obs['images'].cpu(), scene_obs['depths'].cpu(), scene_obs['poses'].cpu(), scene_obs['intrinsics'].cpu()
# print (depth[0])
# print (rgb[0])
# print (pose[0])
print (Ks[0])
print (depth.shape)
print (rgb.shape)
print (pose.shape)
print (Ks.shape)
unprojected = unproject_masked_depth_to_xyz_coordinates(
    # depth = depth[0, None].unsqueeze(1),
    # pose = poses_opencv[0, None],
    depth = depth.unsqueeze(1).squeeze(-1),
    pose = pose,
    inv_intrinsics = torch.linalg.inv(Ks)[:, :3, :3],
    # mask: Optional[torch.Tensor] = None,
) 

ptc = Pointclouds(
    [unprojected.reshape(-1, 3)],
    features = [rgb.reshape(-1,3)],
).subsample(100000)

fig = plot_scene({
    "global scene": dict(
        ptc=ptc
    )
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=200_000,
    height=1000,
    # width=1000,
)

add_camera_poses(fig, pose)
fig.update_layout(
    # width=width,
    height=1000,
    # aspectmode="data"
)